In [ ]:
# Databricks notebook source
# MAGIC %md
# MAGIC # Bronze Backfill - AQI Historical (one-time or ad-hoc)
# MAGIC Loads historical AQI for a date range to seed the Bronze layer.


In [ ]:
# COMMAND ----------
import json
from datetime import datetime, timedelta

import requests
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType


In [ ]:
# COMMAND ----------
# historical API assumed pattern:
# {aqi_history_api_url}?city={city}&start={YYYY-MM-DD}&end={YYYY-MM-DD}&token={token}

dbutils.widgets.text("aqi_history_api_url", "")
dbutils.widgets.text("aqi_api_token", "")
dbutils.widgets.text("cities_csv", "delhi,mumbai,bengaluru")
dbutils.widgets.text("start_date", "2024-01-01")
dbutils.widgets.text("end_date", "2024-12-31")
dbutils.widgets.text("bronze_catalog", "main")
dbutils.widgets.text("bronze_schema", "wattrac_bronze")
dbutils.widgets.text("historical_table", "aqi_historical_raw")

API_URL = dbutils.widgets.get("aqi_history_api_url")
TOKEN = dbutils.widgets.get("aqi_api_token")
CITIES = [c.strip() for c in dbutils.widgets.get("cities_csv").split(",") if c.strip()]
START_DATE = dbutils.widgets.get("start_date")
END_DATE = dbutils.widgets.get("end_date")
BRONZE_CATALOG = dbutils.widgets.get("bronze_catalog")
BRONZE_SCHEMA = dbutils.widgets.get("bronze_schema")
TABLE_NAME = dbutils.widgets.get("historical_table")
TARGET_TABLE = f"{BRONZE_CATALOG}.{BRONZE_SCHEMA}.{TABLE_NAME}"


In [ ]:
# COMMAND ----------
def daterange(start: str, end: str):
    cur = datetime.strptime(start, "%Y-%m-%d").date()
    stop = datetime.strptime(end, "%Y-%m-%d").date()
    while cur <= stop:
        yield cur.isoformat()
        cur += timedelta(days=1)

schema = StructType([
    StructField("city", StringType(), False),
    StructField("observation_date", StringType(), False),
    StructField("avg_aqi", DoubleType(), True),
    StructField("min_aqi", DoubleType(), True),
    StructField("max_aqi", DoubleType(), True),
    StructField("pm25", DoubleType(), True),
    StructField("pm10", DoubleType(), True),
    StructField("no2", DoubleType(), True),
    StructField("so2", DoubleType(), True),
    StructField("co", DoubleType(), True),
    StructField("o3", DoubleType(), True),
    StructField("ingestion_ts", TimestampType(), False),
    StructField("ingestion_date", StringType(), False),
    StructField("raw_payload", StringType(), False),
])


In [ ]:
# COMMAND ----------
rows = []
run_ts = datetime.utcnow()

for city in CITIES:
    for dt in daterange(START_DATE, END_DATE):
        params = {"city": city, "start": dt, "end": dt, "token": TOKEN}
        response = requests.get(API_URL, params=params, timeout=60)
        response.raise_for_status()
        payload = response.json()
        metrics = payload.get("data", {})

        rows.append({
            "city": city,
            "observation_date": dt,
            "avg_aqi": metrics.get("avg_aqi"),
            "min_aqi": metrics.get("min_aqi"),
            "max_aqi": metrics.get("max_aqi"),
            "pm25": metrics.get("pm25"),
            "pm10": metrics.get("pm10"),
            "no2": metrics.get("no2"),
            "so2": metrics.get("so2"),
            "co": metrics.get("co"),
            "o3": metrics.get("o3"),
            "ingestion_ts": run_ts,
            "ingestion_date": run_ts.strftime("%Y-%m-%d"),
            "raw_payload": json.dumps(payload),
        })

hist_df = spark.createDataFrame(rows, schema=schema)


In [ ]:
# COMMAND ----------
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {BRONZE_CATALOG}.{BRONZE_SCHEMA}")
(hist_df.write
 .format("delta")
 .mode("append")
 .partitionBy("ingestion_date")
 .saveAsTable(TARGET_TABLE))

display(hist_df)
print(f"Inserted {hist_df.count()} rows into {TARGET_TABLE}")
